In [1]:
from __future__ import annotations

import pathlib
import re
import sqlite3
import warnings
from os import PathLike

import geopandas as gpd

import config

config.config()

In [2]:
BAG3D_BASE_DATA_URL = "https://data.3dbag.nl/"
BAG3D_VERSION = "v2024.02.28"

In [3]:
def get_version(path: str | bytes | PathLike) -> str:
    # NOTE: A GeoPackage files is an SQLite database with a standardized structure.
    con = sqlite3.connect(path)
    cur = con.cursor()

    # NOTE: The column and table names used in this query are specified by the GeoPackage standard and are guaranteed to be the same regardless of dataset.
    query = "SELECT metadata FROM gpkg_metadata"
    # NOTE: The result of the query is also specified by the standard.
    meta = cur.execute(query).fetchone()[0]

    # NOTE: The versioning scheme used by RoofSense is the same as that of the 3DBAG. See https://docs.3dbag.nl/en/overview/release_notes/ for more information.
    ver_fmt = r"v\d{4}\.\d{2}\.\d{2}"
    ver = re.search(ver_fmt, meta)
    if ver is None:
        raise RuntimeError(f"Failed to infer the file version of the tile index: {path!r}. The file may have not been produced by RoofSens or be corrupt.")
    return ver.group()

In [4]:
from enum import UNIQUE, verify, StrEnum, auto
from collections.abc import Mapping
from typing import Optional


@verify(UNIQUE)
class Index(StrEnum):
    BAG3D = auto()
    IMAGE = auto()
    LIDAR = auto()


def update_indices(src_filepaths: Optional[Mapping[Index, str | bytes | PathLike]] = None,
                   dst_filepaths: Optional[Mapping[Index, str | bytes | PathLike]] = None) -> None:
    # validate inputs
    if src_filepaths is None:
        _src_fpaths = {Index.BAG3D: "tmp/bag3d.gpkg", Index.IMAGE: "tmp/image.gpkg", Index.LIDAR: "tmp/lidar.gpkg", }
    else:
        _src_fpaths = src_filepaths
    
    if dst_filepaths is None:
        _dst_fpaths = {Index.BAG3D: "tmp/bag3d.gpkg", Index.IMAGE: "tmp/image.gpkg", Index.LIDAR: "tmp/lidar.gpkg", }
    else:
        _dst_fpaths = dst_filepaths

    # validate version
    bag3d_fpath = _dst_fpaths[Index.BAG3D]
    if pathlib.Path(bag3d_fpath).is_file():
        ver = get_version(bag3d_fpath)
    else:
        warnings.warn(f"Unable to locate a local copy of the 3DBAG tile index at {bag3d_fpath!r}. The file does not exist.",
                      UserWarning)
        ver = ""

    # update indices
    if ver != BAG3D_VERSION:
        if ver:
            warnings.warn(f"Detected version mismatch between declared ({BAG3D_VERSION}) and existing ({ver}) 3DBAG versions. The tle indices will be updated.",
                          UserWarning)
        _update_bag3d_index(_src_fpaths[Index.BAG3D], _dst_fpaths[Index.BAG3D])
        _update_image_index(_src_fpaths[Index.IMAGE], _dst_fpaths[Index.IMAGE])
        _update_lidar_index(_src_fpaths[Index.LIDAR], _dst_fpaths[Index.LIDAR])


def _update_bag3d_index(src_filepath: str | bytes | PathLike, dst_filepath: str | bytes | PathLike) -> None:
    index_url = f"{BAG3D_BASE_DATA_URL}{BAG3D_VERSION.replace('.', '')}/tile_index.fgb"
    index = gpd.read_file(index_url)
    columns = {"tile_id": "id", "cj_sha256": "sha256", "cj_download": "url", "geometry": "geometry", }
    index = index[columns.keys()].rename(columns=columns)


def _update_image_index(out_dir):
    ...


def _update_lidar_index(out_dir):
    ...

In [6]:
update_indices()

C:\Users\Dimit\AppData\Local\Temp\ipykernel_22452\2261931556.py:6: UserWarning: '' does not exist.
  warnings.warn(f"{out_dir!r} does not exist.", UserWarning)
C:\Users\Dimit\AppData\Local\Temp\ipykernel_22452\2261931556.py:10: UserWarning: Detected version mismatch. Updating indices...
  warnings.warn("Detected version mismatch. Updating indices...", UserWarning)
